NOTES:

   - multiple attachments upwards are 50% ubottu (documentation bot)
       otherwise: seems annotator was unsure of the real target, but probably same cluster anyway -> probbaly very annotator-dependent, do we have that info ? 
   - self-loops starts cluster, but crosses end of previous clusters (how much overlap?)
   - isolated self-loop probably its own class (short/smiley/etc)
   - some turns with no head acts as self-loops/beginning of clusters
   - server always self-loop ? -> no cluster


In [1]:
import itertools
import sys
import logging
from typing import Any, Tuple, Dict, List, Iterable
import torch
import torch.optim as optim

In [2]:
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, LabelField, TextField, ListField, SequenceLabelField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, PretrainedBertIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer, PretrainedTransformerTokenizer
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding, PretrainedBertEmbedder
from allennlp.modules.seq2vec_encoders import BertPooler

from allennlp.modules import Seq2VecEncoder, Seq2SeqEncoder
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.data.vocabulary import Vocabulary

from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator
from allennlp.common import Params
from allennlp.nn import util

In [3]:
%load_ext autoreload
%autoreload 2

In [6]:
#from dummy_chat_reader import ChatReader
from irc_chat_reader import ChatReader

In [7]:
token_indexers = {"tokens": SingleIdTokenIndexer()}

tokenizer_cfg = Params({"word_splitter": {"language": "en"}})

word_tokenizer = Tokenizer.from_params(tokenizer_cfg)

In [8]:
reader = ChatReader(
    tokenizer=word_tokenizer,
    token_indexers=token_indexers,
    raw = True,
    sub_sequence = None,
    #clip = 200
    )
train_instances = reader.read("../data/train")
dev_instances = reader.read("../data/dev")

153it [00:28,  5.74it/s]
10it [00:00, 11.80it/s]


In [9]:
vocab = Vocabulary.from_instances(train_instances+dev_instances)

100%|██████████| 163/163 [00:00<00:00, 323.79it/s]


In [34]:
#for i in train_instances:
#    #print(i)
#    i["lines"].index(vocab)
#    i["arcs"].index(vocab)
#    print(i["lines"].get_padding_lengths())

In [8]:
i0 = train_instances[0]
i0["is_server"].array[:10]

array([False, False, False, False, False, False, False, False, False,
       False])

In [36]:
#while True:
#    pass

In [37]:
#sum([len([x for x in i["metadata"].metadata["addressees"] if x is None]) for i in train_instances])

In [9]:
c = 0
for n,i in enumerate(train_instances):
    nbarcs = len(i["arcs"].indices)
    if nbarcs==0: 
        c+=1 
        print(n)
print("nb of document with zeros edges:",c)

nb of document with zeros edges: 0


In [39]:
#distance max
#for instance in train_instances:
#    maxl = max([abs(j-i) for (i,j) in instance["arcs"].indices])
#    print(maxl)
               

In [44]:
from allennlp.models import Model
from typing import Dict, List, Iterable
from allennlp.modules import TimeDistributed
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

In [10]:
dev_instances = reader.read("../data/dev")

10it [00:00, 11.79it/s]


In [12]:
# probing the gold annotations to see how many multiple attachements
def store_gold(instances,chat_id):
    gold = {}
    for (i,j) in instances[chat_id]["arcs"].indices:
        if i in gold:
            gold[i].append(j)
        else:
            gold[i] = [j]
    return gold

In [13]:
dev_gold = {i:store_gold(dev_instances,i) for i in range(len(dev_instances))}

In [14]:
chat_id = 0

In [54]:
# check multiple attachments
from collections import Counter
c = Counter([len(dev_gold[chat_id][i]) for i in dev_gold[chat_id]])
c

Counter({2: 13, 1: 233, 3: 1})

In [139]:
# print users for turns with multiple heads
all = []
for chat_id in range(len(dev_instances)):
    print("------",chat_id)
    for i in dev_gold[chat_id]: 
        if len(dev_gold[chat_id][i])>1:
            all.append((dev_instances[chat_id]["lines"][i].tokens[1]))
            print(all[-1])
print("total",len(all))        
Counter(all).most_common(10)

------ 0
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
ubottu
tstebut
------ 1
ubottu
edbian
ubottu
ubottu
toad
------ 2
ud
phill
CPayan
phill
------ 3
ubottu
ubottu
eepberries
Nytrix
ubottu
ubottu
b1n42y
stevr1it
ubottu
ubottu
Silicium
Nytrix
ubottu
Nytrix
ubottu
ActionParsnip
rogerio
------ 4
corba
ubottu
ubottu
ubottu
corba
MonkeyDust
figure002
groob
------ 5
L1nuxRules
ubottu
bonhoffer
bonhoffer
OttScorp
bonhoffer
------ 6
allsystemsarego
ActionParsnip1
ubottu
ActionParsnip1
ubottu
rookeeeeeeeee
sinuc
sken
sinuc
sken
sken
sken
ubottu
ubottu
ubottu
BoogieBoo
Panarchy
BoogieBoo
------ 7
ubottu
ubottu
ubottu
Frantic
optiks
optiks
ubottu
ubottu
ubottu
ubottu
------ 8
nubbe
microhaxo
microhaxo
bob2
bob2
microhaxo
lukus001
xabbu|
CDK
microhaxo
floo
------ 9
jasoncohen
Raf99
f_newton
MartenH
Nevado
linlin
jasoncohen
MartenH
f_newton
CircleofChaos
f_newton
Agamotto
total 105


[(ubottu, 40),
 (sken, 4),
 (microhaxo, 4),
 (Nytrix, 3),
 (bonhoffer, 3),
 (f_newton, 3),
 (phill, 2),
 (corba, 2),
 (ActionParsnip1, 2),
 (sinuc, 2)]

In [45]:
def display_chat(instances,gold,chat_id,start=0,nb=250,file=sys.stdout):
    turn_key = "lines"
    for i,turn in enumerate(instances[chat_id][turn_key][start:start+nb]):
        k = i + start
        head = gold[chat_id].get(k,"NONE")
        if head == [k]: head = "SELF-LOOP"
        print(k,"->",head,turn.tokens,file=file)

In [21]:
display_chat(dev_instances,dev_gold,chat_id)

0 -> NONE [<, elad, `, >, I, just, got, whatever, got, installed, by, firefox, when, I, first, entered, a, webpage, with, flash, content, .]
1 -> NONE [<, ab2qik, >, fccf, ,, Yes]
2 -> NONE [<, tavelram, >, genii, ,, ok, ,, there, was, nt, a, 1.5, of, rdesktop, ,, and, I, think, I, solved, my, original, problem, by, switching, to, oss, instead, of, alsa, ,, but, thanks, anyway, .]
3 -> [0] [<, thiebaude, >, !, flash]
4 -> [3, 0] [<, ubottu, >, To, install, Flash, see, #, http, -, See, also, !, Restricted, and, !, Gnash]
5 -> [2] [<, genii, >, tavelram, :, np]
6 -> [1] [<, fccf, >, ab2qik, :, please, pastebin, your, fstab, ..., I, 'll, take, a, look]
7 -> [6] [<, ab2qik, >, fccf, ,, thanks]
8 -> SELF-LOOP [<, xTEMPLARx, >, :D]
9 -> [6] [<, ab2qik, >, fccf, ,, sorry, that, s, difficult, since, i, just, get, a, shell]
10 -> SELF-LOOP [<, frog, _, >, hi, ,, i, live, in, stud, ., house, ,, i, am, connected, to, swich, and, the, whole, stud, ., house, is, conecctded, to, dsl, ., how, can, i,

In [43]:
SAVE=FALSE

if SAVE:
    with open("log.chat","w") as f_out:
        for chat_id in range(len(dev_instances)):
            print("==============DEV CHAT nb %s ======="%chat_id,file=f_out)
            display_chat(dev_instances,dev_gold,chat_id,file=f_out)

In [23]:
#sorted(dev_instances[0]["arcs"].indices)

In [141]:
from IPython.display import HTML as html_print

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

map = ["green","blue","yellow","red","gray",
       "magenta","olive","purple","lightblue","salmon",
       "orange","teal","tomato","black"]


def print_color(s,color="black"):
    display(html_print(cstr(s,color=color)))

def gen_color(map,nmax=30):
    i = 0
    c = 0
    nb = len(map)
    while c<nmax:
        yield map[i]
        i = i + 1
        if i==nb:
            i=0  
        c = c + 1
        
        
colors = list(gen_color(map,nmax=250))
#colors

In [142]:
#for i in gen_color(map): print_color("test",color=i)

In [155]:
def display_clusters(dev_instances,dev_gold,chat_id = 0):
    clusters = {}
    cid = -1
    result = []

    for i,turn in enumerate(dev_instances[chat_id]["lines"]):
        head = dev_gold[chat_id].get(i,"None")
        user = turn.tokens[1]
        auto = (head[0]==i) # self-loop
        #print(i,head)
        if True:
            if auto or head=="None":
                cid += 1
                clusters[i] = cid
                if auto: 
                    symbol = "X"
                else:
                    symbol = "x"
            else:
                clusters[i] = clusters[head[0]]
                symbol = "■"
            color = colors[clusters[i]]
            result.append(cstr(symbol,color=color))
            if i%50==0:
                result.append("\n")
    print(len(set(clusters.values())),"found")
    return clusters, html_print("".join(result))


In [156]:
clusters, figure = display_clusters(dev_instances,dev_gold,1)
figure

44 found


In [157]:
clusters, figure = display_clusters(dev_instances,dev_gold,2)
figure

66 found


In [153]:
display_chat(dev_instances,dev_gold,2)

0 -> SELF-LOOP [<, LinuxJones, >, night, all, :)]
1 -> SELF-LOOP [_, server, yohannes, [, ~yohannes@adsl-67, -, 112, -, 218, -, 21.dsl.sntc01.pacbell.net, ], has, joined, #, ubuntu]
2 -> SELF-LOOP [<, yohannes, >, can, anyone, recommend, any, app, to, create, /, open, *, .rar, file, ?]
3 -> [2] [<, Hikaru79, >, yohannes, ,, why, not, WinRAR, ?]
4 -> [3] [<, Hikaru79, >, #, http]
5 -> [3] [<, yohannes, >, i, am, trying, to, weight, my, option, ., any, other, apps, ?]
6 -> [5] [<, Hikaru79, >, Just, download, RAR, 3.41, for, Linux]
7 -> [5] [<, Hikaru79, >, Oh]
8 -> [7] [<, Hikaru79, >, I, see]
9 -> [8] [<, Hikaru79, >, Only, one, I, know, of, ,, sorry, =(]
10 -> [6] [<, yohannes, >, btw, ,, how, can, i, install, that, darn, thing, ?, :)]
11 -> [10] [<, yohannes, >, i, downloaded, winrar, ,, does, not, have, any, instruction, to, install]
12 -> [5] [<, Nafallo, >, file, -, roller]
13 -> [11] [<, yohannes, >, i, suspect, i, need, to, do, something, with, the, makefile, included]
14 -> SEL